# Ejemplo Web scrapping 2: Noticias deportivas

En este notebook vamos a ver otro ejemplo de web scrapping, este con noticias deportivas:

## Importando librerías

Al igual que en el notebook anterior, importamos lo necesario:

In [39]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from urllib.request import urlopen

Leemos la web:

In [40]:
# Cabeceras:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
# url web a scrapear:
marca = "https://www.marca.com/"

response = get(marca, headers = headers)

Hacemos el get:

In [41]:
print(response) # un 200 es una buena señal :D

<Response [200]>


Todo bien, pasamos a sacar cosas:

In [42]:
html_soup = BeautifulSoup(response.text, 'html.parser')

Tras enalizar la web, nos hemos fijado que todas las noticias tienen etiquetas ``h2`` y clase ``mod-title``:

In [43]:
noticias = html_soup.find_all('h2', class_='mod-title')
noticias[0]

<h2 class="mod-title" itemprop="headline"><a href="https://www.marca.com/futbol/copa-rey/alcoyano-vs-r-madrid/cronica/2021/01/20/6008b14c46163f85208b45f5.html" itemprop="url" title="¡Qué bochorno de Madrid!"> ¡Qué bochorno de Madrid!
</a></h2>

Dentro de cada una de ellas, tenemos que extraer, a su vez, el ``title`` de las ``a``:

In [44]:
noticia_ejemplo = noticias[0].find_all("a")[0]
# Con el strip eliminamos todas aquellas cosas raras al comienzo y final del string:
noticia_ejemplo.text.strip()

'¡Qué bochorno de Madrid!'

También podemos sacar el enlace como:

In [45]:
noticia_ejemplo.get_attribute_list('href')[0]

'https://www.marca.com/futbol/copa-rey/alcoyano-vs-r-madrid/cronica/2021/01/20/6008b14c46163f85208b45f5.html'

Ahora podemos acceder a ese link y extraer la información de la noticia en sí:

In [46]:
url_2 = noticia_ejemplo.get_attribute_list('href')[0]

response_2 = get(url_2, headers = headers)
html_soup_2 = BeautifulSoup(response_2.text, 'html.parser')


# for texto in html_soup_2.find_all("p"):
#     print(texto.text)

# El texto que queremos está en los p y en los h3, pero también se nos está colando cosas que no nos resultan neceasrias, ¿cómo podemos proceder?
# Pues leyendo todo junto (que estará en orden según la página) y quedándonos con el texto hasta que detectemos algo que nos haga parar.
# En este caso, ese algo será cuando lleguemos a un texto 'h3' que diga "Temas relaciondos". A partir de ese momento, ya no nos interesan 
#ni los 'p' ni los 'h3'
texto_noticia = html_soup_2.find_all(["p", "h3"])
texto_noticia

noticia_completa = []
cont = 0
for line in texto_noticia:
    cont += 1
    if line.text == 'Temas relacionados':
        break
    # Nos saltamos siempre las 2 primeras líneas:
    if cont > 2:
        noticia_completa.append(line.text.strip())

# noticia_completa
# Y lo juntamos todo en un solo texto:
''.join(noticia_completa)

'Estadísticas del partido | Así lo vivimosEl Real Madrid vivió una noche negra. El Alcoyano entra en la historia del Madrid como antes lo hicieron el Alcorcón o el Toledo, colocando al conjunto blanco en una crisis indiscutible. La noche mágica del meta José Juan, el esfuerzo del bloque y el acierto indiscutible en los momentos decisivos dieron una victoria con resonancia en todo el planeta fútbol, por su hazaña y por las consecuencias que pueda tener. La impotencia y la falta de reacción del equipo de Zidane pone aEl Real Madrid compareció en El Collao, el depósito nacional de la moral futbolera, repleto de incógnitas en el juego -los resultados lo demuestran- y en el vestuario - la inminente salida de Odegaard y la de Jovic pone en cuestión el manejo de Zidane con las promesas blancas-. Así que el técnico francés dio orden de no especular en Alcoy, presentando un once con más habituales de los esperados. A pesar de la rotación masiva en defensa, en parte obligada por las lesiones de 

In [47]:
texto_noticia

[<p class="edition-selector__menu-title">Ediciones:</p>,
 <p class="tab-news-title js-newsTitle">Futbol/copa-rey</p>,
 <p class="summary-lead second izquierda"><i class="item-sumary-lead"></i><a href="https://www.marca.com/futbol/copa-del-rey/alcoyano-r-madrid-directo/2021/01/20/01_0105_20210120_2501_186.html">Estadísticas del partido | Así lo vivimos</a></p>,
 <p><span class="capital-letter">E</span>l Real Madrid vivió una noche negra. El Alcoyano entra en la historia del Madrid como antes lo hicieron el Alcorcón o el Toledo, colocando al conjunto blanco en una crisis indiscutible. La noche mágica del meta José Juan, el esfuerzo del bloque y el acierto indiscutible en los momentos decisivos dieron una victoria con resonancia en todo el planeta fútbol, por su hazaña y por las consecuencias que pueda tener. La impotencia y la falta de reacción del equipo de Zidane pone a </p>,
 <p>El Real Madrid compareció en El Collao, el depósito nacional de la moral futbolera, repleto de incógnitas e

Iteramos con esta condición. Y para combinarlo con lo otro, podríamos extraer todos los links a noticias y, posteriormente, extraer el texto de cada una de ellas:

In [48]:
noticias[0].find_all("a")[0].text.strip()

'¡Qué bochorno de Madrid!'

In [49]:
# Leemos títulos de artículos y sus links y los guardamos:
marca = "https://www.marca.com/"

response = get(marca, headers = headers)

html_soup = BeautifulSoup(response.text, 'html.parser')

noticias = html_soup.find_all('h2', class_='mod-title')

# Noticia a noticia para sacar links:
textos = []
links = []
for noticia in noticias:
    try:
        text_noticia = noticia.find_all("a")[0].text.strip()
        link = noticia.find_all("a")[0].get_attribute_list('href')[0]
        
        textos.append(text_noticia)
        links.append(link)
    except:
        pass
    
# Una vez tenemos las noticias, iteramos por cada uno de sus enlaes:
textos_noticias_completos = []
for link in links:
    response_2 = get(link, headers = headers)
    html_soup_2 = BeautifulSoup(response_2.text, 'html.parser')
    
    texto_noticia = html_soup_2.find_all(["p", "h3"])
    texto_noticia
    noticia_completa = []
    cont = 0
    for line in texto_noticia:
        cont += 1
        if line.text == 'Temas relacionados':
            break
        # Nos saltamos siempre las 2 primeras líneas:
        if cont > 2:
            noticia_completa.append(line.text.strip())
    
    textos_noticias_completos.append(''.join(noticia_completa))

Finalmente, nos creamos el dataframe con todo lo que tenemos.

Importante, si vamos a manejar muchos datos (pero muchos muchos), será mejor ir creando el dataframe poco a poco (cada x llamadas) para no almacenar demasiada información duplicada al mismo tiempo.

In [50]:
cols = ['Título', 'Descripción', 'Enlace']

noticias_deportivas = pd.DataFrame({'Título': textos,
                                    'Descripción': textos_noticias_completos,
                                    'Enlace': links})[cols]

noticias_deportivas.to_excel('marca_noticias.xls', index=False)
noticias_deportivas_df = pd.read_excel('marca_noticias.xls')

In [51]:
noticias_deportivas_df

,Título,Descripción,Enlace
0,¡Qué bochorno de Madrid!,Estadísticas del partido | Así lo vivimosEl Re...,https://www.marca.com/futbol/copa-rey/alcoyano...
1,"Zidane: ""Esto no es una vergüenza, puede pasar""",Contracrónica: Esto sí puede acabar con Zidane...,https://www.marca.com/futbol/copa-del-rey/alco...
2,"Si Zidane se salva de esta, es un milagro",La crónica del Alcoyano-MadridEl uno a uno del...,https://www.marca.com/futbol/copa-rey/2021/01/...
3,Así se entra en la historia negra: la culpa es...,"Alcoyano-Real Madrid, en directoOdeggard se ac...",https://www.marca.com/futbol/real-madrid/2021/...
4,Así la clasificación de LaLiga,© Enero\n2021\nUnidad Editorial Información De...,https://www.marca.com/futbol/primera-division/...
5,El Mercedes Clase G finaliza en una excelente ...,El Dakar 2021 ya es historia. El Raid más duro...,https://www.marca.com/uestudio/2021/01/18/6005...
6,"Odegaard, cerca de la Real",Martin Odegaard lo tiene claro y sigue firme e...,https://www.marca.com/futbol/real-madrid/2021/...
7,'No' a Koeman por Eric García,El Govern aprueba el voto por correo para las ...,https://www.marca.com/futbol/barcelona/2021/01...
8,El Getafe de Kubo cambia de cara,Estadísticas del partido y así lo vivimosVeloc...,https://www.marca.com/futbol/primera-division/...
9,El último servicio de Willian José,Estadísticas del partido y así lo vivimosWilli...,https://www.marca.com/futbol/copa-rey/cordoba-...
